In [1]:
import math
from scipy.optimize import minimize
import numpy as np
from matplotlib import pyplot as plt

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
bounds = [(0.01, 10), (0.01, 10), (0.01, 10), (0.01, 10),(0.01, 10), 
          (0.01, 10), (0.01, 10), (0.01, 10), 
          (0.01, 10), (0.01, 10), (0.01, 10), (0.01, 10),(0.01, 10),
          (0.01, 10), (0.01, 10), (0.01, 10), 
          (0.01, 10), (0.01, 10), (0.01, 10), (0.01, 10)]
x_opt = [3.16246061572185, 3.12833142812967, 3.09479212988791, 3.06145059523469, 3.02792915885555, 2.99382606701730,
         2.95866871765285, 2.92184227312450, 0.49482511456933, 0.48835711005490, 0.48231642711865, 0.47664475092742,
         0.47129550835493, 0.46623099264167, 0.46142004984199, 0.45683664767217, 0.45245876903267, 0.44826762241853,
         0.44424700958760, 0.44038285956317]
y_opt = -0.80361910412559


def G02(X):
    n = 20
    up_left, down = 0.0, 0.0
    up_right = 1
    for i in range(n):
        up_left += (math.cos(X[i])) ** 4
        up_right *= (math.cos(X[i])) ** 2
        down += (i + 1) * X[i] * X[i]
    y = -abs((up_left - 2 * up_right) / math.sqrt(down))
    x_mul = 1
    x_sum = 0.0
    for i in range(n):
        x_mul *= X[i]
        x_sum += X[i]
    g1 = 0.75 - x_mul
    g2 = x_sum - 7.5 * n

    return y, g1, g2

In [4]:
# y = lambda X : G02(X)[0]
# x0 = [0.01 for i in range(20)]
# # x0 = [5 for i in range(20)]
# # x0 = [10 for i in range(20)]
# minfunction(y,x0,bounds)

In [5]:
# pN = 1000
# dim = 20
# fmax = 124211
# vmax = 50.75
# keth = 0.5
# eta = 0.5
# X = np.zeros((pN, dim))
# for i in range(pN):
#     for j in range(dim):
#         X[i][j] = bounds[j][0]+(bounds[j][1]-bounds[j][0])*np.random.random()

# fs = []    
# Vs = []
# for i in range(pN):
#     for j in range(pN):
#         if i != j:
#             deltaf = G02(X[i])[0] - G02(X[j])[0]
#             delraV = np.sum(G02(X[i])[1:3])-np.sum(G02(X[j])[1:3])
#             fs.append(deltaf)
#             Vs.append(delraV)

# plt.scatter(fs,Vs,s=1)
# plt.axhline(y=vmax, xmin=-fmax, xmax=fmax)
# plt.axhline(y=-vmax, xmin=-fmax, xmax=fmax)
# plt.axhline(y=vmax*keth, xmin=-fmax, xmax=fmax)
# plt.axhline(y=-vmax*keth, xmin=-fmax, xmax=fmax)
# plt.axhline(y=vmax*keth*eta, xmin=-fmax, xmax=fmax)
# plt.axhline(y=-vmax*keth*eta, xmin=-fmax, xmax=fmax)

# plt.axvline(x=fmax, ymin=-vmax, ymax=vmax)
# plt.axvline(x=-fmax, ymin=-vmax, ymax=vmax)
# plt.axvline(x=fmax*eta, ymin=-vmax, ymax=vmax)
# plt.axvline(x=-fmax*eta, ymin=-vmax, ymax=vmax)
# plt.show()

In [6]:
### 适应度函数
def fitnessG02_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 0
    fmin = -124.211
    epsion = 1.5
    count = np.sum(np.array(G02(X))[1:3]>0)
    
    return -(G02(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG02_vio(X):
    """
    
    """
    fmax = 0
    fmin = -124.211
    vmax = 50.75
    e = 0.1
    n=9
    
    res = G02(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:3]>0)
    
    for i in range(1,len(res)):
        if count > 0:
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
            
    return fitness  

In [7]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-10000.0)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-10000.0)    # 全局最佳适应值
        
        self.gbest_1 = np.zeros((self.pN, self.dim)) # 第5000次迭代时的最优解
        self.g_fit_1 = np.full(self.pN,-10000.0) # 第5000次迭代时的最优适应度值
        self.gbest_2 = np.zeros((self.pN, self.dim)) # 第50000次迭代时的最优解
        self.g_fit_2 = np.full(self.pN,-10000.0) # 第50000次迭代时的最优适应度值
        self.gbest_3 = np.zeros((self.pN, self.dim)) # 第500000次迭代时的最优解
        self.g_fit_3 = np.full(self.pN,-10000.0) # 第500000次迭代时的最优适应度值
        self.FES = 500000   # 到达最优值需要的最小迭代次数
        self.devi = np.full(max_iter,0.0)  ##每次迭代完成最优解与实际最优解目标函数的偏差
        self.fe = 0
        self.su = 0
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
    
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG02_num(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/1500))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
#             w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            w = 0.85
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG02_num(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/1500))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
                        
            if t == 5000-1:
                self.gbest_1 = self.gbest
                self.g_fit_1 = self.g_fit
            if t == 50000-1:
                self.gbest_2 = self.gbest
                self.g_fit_2 = self.g_fit
            if t == 500000-1:
                self.gbest_3 = self.gbest
                self.g_fit_3 = self.g_fit
            best_index = np.argmax(self.g_fit)
            best_solution = self.gbest[best_index]
            de = G02(best_solution)[0] - y_opt                    ###
            self.devi[t] = de
            if np.sum(np.array(G02(best_solution)[1:3])>0) == 0:  ###
                self.fe = 1
            if de <= 0.0001:
                self.FES = t
                self.su = 1
            print(self.g_fit)  
            
        best_index = np.argmax(self.g_fit)
        best_solution = self.gbest[best_index]

        return self.gbest_1, self.g_fit_1, self.gbest_2, self.g_fit_2, self.gbest_3, self.g_fit_3, self.FES, self.devi, self.fe, self.su

In [ ]:
def statistic(solutions,fitnesses):
    """
    给定25次的最优解和最优适应度值求统计值
    solutions：25次最优解
    fitnesses：25次最优适应度值
    """
    ress = []
    for i in range(len(fitnesses)):
        res = G02(solutions[i])[0]                    ###
        ress.append(res)
    mean = np.mean(ress)
    std = np.std(ress)
    
    best_index = np.argmax(fitnesses)
    best_solution = solutions[best_index]
    best_res = G02(best_solution)                         ##
    best_function = best_res[0] 
    best_vio_num = np.sum(np.array(best_res[1:3])>0)     ##

    worst_index = np.argmin(fitnesses)
    worst_solution = solutions[worst_index]
    worst_res = G02(worst_solution)                       ##
    worst_function = worst_res[0]
    worst_vio_num = np.sum(np.array(worst_res[1:3])>0)   ##

    median = np.median(fitnesses)
#     print(median)
    median_index = np.argwhere(fitnesses==median)[0][0]
#     print(median_index)
    median_solution = solutions[median_index]
    median_res = G02(median_solution)                      ##
    median_function = median_res[0]
    median_vio_num = np.sum(np.array(median_res[1:3])>0)  ##
    
    median_vio_vio_mean = 0.0
    flag = 0
    c1 = 0
    c2 = 0
    c3 = 0
    for i in range(1,3):                                  ###
        if median_res[i] > 0:
            flag = flag + 1
            median_vio_vio_mean = median_vio_vio_mean + median_res[i]
        if median_res[i] > 1.0:
            c1 = c1 + 1
        if median_res[i] > 0.01:
            c2 = c2 + 1
        if median_res[i] > 0.0001:
            c3 = c3 + 1
    if flag != 0:
        median_vio_vio_mean = median_vio_vio_mean / flag
    
    return best_function, best_vio_num, worst_function, worst_vio_num, median_function, median_vio_num, median_vio_vio_mean, c1, c2, c3, mean, std

In [ ]:
def st(fes):
    best = np.min(fes)
    median = np.median(fes)
    worst = np.max(fes)
    mean = np.mean(fes)
    std = np.std(fes)
    
    return best, median, worst, mean, std

In [8]:
PSO = PSO_fit(pN=15000, dim=20, max_iter=500000, bounds=bounds)

In [ ]:
solutions1 = []
fitnesses1 = []
solutions2 = []
fitnesses2 = []
solutions3 = []
fitnesses3 = []
fes = []
fs = 0        ## 可行率
sus = 0       ## 成功率

for i in range(24):
    
    PSO.init_Population()
    gbest_1, g_fit_1, gbest_2, g_fit_2, gbest_3, g_fit_3, FES, devi, fe, su = PSO.iterator()
    
    np.save("./results/G02/FES_3/gbest_" + str(i) + "_1.npy", gbest_1)      ###
    np.save("./results/G02/FES_3/g_fit_" + str(i) + "_1.npy", g_fit_1)      ###
    np.save("./results/G02/FES_4/gbest_" + str(i) + "_2.npy", gbest_2)      ###
    np.save("./results/G02/FES_4/g_fit_" + str(i) + "_2.npy", g_fit_2)      ###
    np.save("./results/G02/FES_5/gbest_" + str(i) + "_3.npy", gbest_3)      ###
    np.save("./results/G02/FES_5/g_fit_" + str(i) + "_3.npy", g_fit_3)      ###
    
    np.save("./results/G02/FES_" + str(i) + ".npy", FES)                    ###
    np.save("./results/G02/devi_" + str(i) + ".npy", devi)                   ###
    
    fes.append(FES)
    
    fs = fs + fe
    sus = sus + su
    
    best_index1 = np.argmax(g_fit_1)
    solutions1.append(gbest_1[best_index1])
    fitnesses1.append(g_fit_1[best_index1])
    
    best_index2 = np.argmax(g_fit_2)
    solutions2.append(gbest_2[best_index2])
    fitnesses2.append(g_fit_2[best_index2])
    
    best_index3 = np.argmax(g_fit_3)
    solutions3.append(gbest_3[best_index3])
    fitnesses3.append(g_fit_3[best_index3])

np.save("./results/G02/FES_3/solutions.npy", solutions1)     ###
np.save("./results/G02/FES_3/fitnesses.npy", fitnesses1)     ###

np.save("./results/G02/FES_4/solutions.npy", solutions2)     ###
np.save("./results/G02/FES_4/fitnesses.npy", fitnesses2)     ###

np.save("./results/G02/FES_5/solutions.npy", solutions3)     ###
np.save("./results/G02/FES_5/fitnesses.npy", fitnesses3)     ###

np.save("./results/G02/fes.npy", fes)                        ###

result0 = st(fes)
fs = fs / 24
sus = sus / 24

result1 = statistic(solutions1, fitnesses1)   
result2 = statistic(solutions2, fitnesses2)
result3 = statistic(solutions3, fitnesses3)

np.savetxt("results/G02/result0.txt", result0)
np.savetxt("results/G02/result1.txt", result1)
np.savetxt("results/G02/result2.txt", result2)
np.savetxt("results/G02/result3.txt", result3)
np.savetxt("results/G02/fs.txt", fs)
np.savetxt("results/G02/sus.txt", sus)